In [67]:
import pandas as pd

corpus = [
    "Apple Apple Banana",
    "Banana Mango Banana",
    "Cherry Cherry Strawberries",
    "Grapes Grapes Strawberries Grapes",
    "Apple Banana Mango",
    "Blueberries Strawberries Apple",
    "Apple Banana Mango",
    "Grapes Grapes Grapes",
    "Blueberries Apple Strawberries",
    "Apple Banana Apple",
    "Cherry Cherry Mango Cherry",
    "Blueberries Strawberries Cherry",
]

dataset = [
    {"query": "apple banana", "relevant_doc_ids": [0, 4, 6, 9]},
    {"query": "grapes", "relevant_doc_ids": [3, 7]},
    {"query": "banana mango", "relevant_doc_ids": [1, 4, 6, 10]},
    {"query": "Cherry", "relevant_doc_ids": [2, 10, 11]},
    {"query": "apple", "relevant_doc_ids": [0, 4, 6, 8, 9]},
    {"query": "Blueberries Strawberries", "relevant_doc_ids": [5, 8, 11]}
]

#simulation of retrieved documents
# retrieved_docs = [
#     [0, 3, 6, 11, 9],
#     [2, 5, 4, 9, 10], # all wrong
#     [2, 4, 1, 10, 9], # first and third one is incorrect
#     [2, 5, 11, 10, 7], # middle one is incorrect
#     [0, 6, 3, 8, 10], # last one is incorrect, 6 and 4 swapped
#     [8, 5, 11, 7, 2] # all correct
# ]

retrieved_docs = [
    [0, 3, 6, 11, 9, 1, 2, 5, 7, 8],  # Retrieved docs for the first query
    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],  # Retrieved docs for the second query
    [2, 4, 6, 7, 9, 0, 1, 3, 5, 8],  # Retrieved docs for the third query
    [5, 1, 2, 6, 0, 4, 7, 3, 8, 9],  # Retrieved docs for the fourth query
    [0, 1, 6, 7, 8, 10, 2, 3, 4, 5],  # Retrieved docs for the fifth query
    [2, 3, 4, 5, 6, 7, 0, 1, 8, 9]   # Retrieved docs for the sixth query
]


df = pd.DataFrame(dataset)
df['retrieved_doc_ids'] = retrieved_docs
df

,query,relevant_doc_ids,retrieved_doc_ids
0,apple banana,"[0, 4, 6, 9]","[0, 3, 6, 11, 9, 1, 2, 5, 7, 8]"
1,grapes,"[3, 7]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]"
2,banana mango,"[1, 4, 6, 10]","[2, 4, 6, 7, 9, 0, 1, 3, 5, 8]"
3,Cherry,"[2, 10, 11]","[5, 1, 2, 6, 0, 4, 7, 3, 8, 9]"
4,apple,"[0, 4, 6, 8, 9]","[0, 1, 6, 7, 8, 10, 2, 3, 4, 5]"
5,Blueberries Strawberries,"[5, 8, 11]","[2, 3, 4, 5, 6, 7, 0, 1, 8, 9]"


In [68]:
K = 3

def success(relevant_doc_ids: list[list[int]], retrieved_doc_ids: list[list[int]], K):
    """ 
    is there any doc of y_pred in y[:K]  
    """
    result = [0] * len(relevant_doc_ids)
    for i in range(len(relevant_doc_ids)):
        for j in retrieved_doc_ids[i]:
            if j in relevant_doc_ids[i][:K]:
                result[i] = 1
                break

    return result

result = success(df['relevant_doc_ids'], df['retrieved_doc_ids'], K)
print(f"Result: {result}")

df[f"success@{K}"] = result
df


Result: [1, 1, 1, 1, 1, 1]


,query,relevant_doc_ids,retrieved_doc_ids,success@3
0,apple banana,"[0, 4, 6, 9]","[0, 3, 6, 11, 9, 1, 2, 5, 7, 8]",1
1,grapes,"[3, 7]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]",1
2,banana mango,"[1, 4, 6, 10]","[2, 4, 6, 7, 9, 0, 1, 3, 5, 8]",1
3,Cherry,"[2, 10, 11]","[5, 1, 2, 6, 0, 4, 7, 3, 8, 9]",1
4,apple,"[0, 4, 6, 8, 9]","[0, 1, 6, 7, 8, 10, 2, 3, 4, 5]",1
5,Blueberries Strawberries,"[5, 8, 11]","[2, 3, 4, 5, 6, 7, 0, 1, 8, 9]",1


In [69]:
from numpy import mean

def mean_reciprocal_rank(relevant_doc_ids: list[list[int]], retrieved_doc_ids: list[list[int]], K):
    """
    mean of (1 / rank of first relevant doc) 
    """
    result = [0] * len(relevant_doc_ids)
    for i in range(len(relevant_doc_ids)):
        first_doc_id = relevant_doc_ids[i][0]
        for j, doc_id in enumerate(retrieved_doc_ids[i][:K]):
            if doc_id == first_doc_id:
                result[i] = 1/(j+1)

    return result, round(float(mean(result)), 3)


result, mrr = mean_reciprocal_rank(df['relevant_doc_ids'], df['retrieved_doc_ids'], K)
print(f"Result: {result}\nMRR: {mrr}")

df[f"mrr@{K}"] = result
df

Result: [1.0, 0, 0, 0.3333333333333333, 1.0, 0]
MRR: 0.389


,query,relevant_doc_ids,retrieved_doc_ids,success@3,mrr@3
0,apple banana,"[0, 4, 6, 9]","[0, 3, 6, 11, 9, 1, 2, 5, 7, 8]",1,1.000000
1,grapes,"[3, 7]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]",1,0.000000
2,banana mango,"[1, 4, 6, 10]","[2, 4, 6, 7, 9, 0, 1, 3, 5, 8]",1,0.000000
3,Cherry,"[2, 10, 11]","[5, 1, 2, 6, 0, 4, 7, 3, 8, 9]",1,0.333333
4,apple,"[0, 4, 6, 8, 9]","[0, 1, 6, 7, 8, 10, 2, 3, 4, 5]",1,1.000000
5,Blueberries Strawberries,"[5, 8, 11]","[2, 3, 4, 5, 6, 7, 0, 1, 8, 9]",1,0.000000


In [70]:
def precision(relevant_doc_ids: list[list[int]], retrieved_doc_ids: list[list[int]], K):
    """ 
    no. of relevant docs in top K retrieved docs / K
    intuition: precision is high when there are more relevent docs in , low when there are irrelevant documents in topK retrieved.
    """
    result = [0] * len(relevant_doc_ids)
    for i in range(len(relevant_doc_ids)):
        rel_docs_count = 0
        for doc_id in retrieved_doc_ids[i][:K]:
            if doc_id in relevant_doc_ids[i]:
                rel_docs_count += 1
        result[i] = rel_docs_count/K

    return result, round(float(mean(result)), 3)


result, prec = precision(df['relevant_doc_ids'], df['retrieved_doc_ids'], K)
print(f"Result: {result}\nPrecision: {prec}")

df[f"precision@{K}"] = result
df

Result: [0.6666666666666666, 0.0, 0.6666666666666666, 0.3333333333333333, 0.6666666666666666, 0.0]
Precision: 0.389


,query,relevant_doc_ids,retrieved_doc_ids,success@3,mrr@3,precision@3
0,apple banana,"[0, 4, 6, 9]","[0, 3, 6, 11, 9, 1, 2, 5, 7, 8]",1,1.000000,0.666667
1,grapes,"[3, 7]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]",1,0.000000,0.000000
2,banana mango,"[1, 4, 6, 10]","[2, 4, 6, 7, 9, 0, 1, 3, 5, 8]",1,0.000000,0.666667
3,Cherry,"[2, 10, 11]","[5, 1, 2, 6, 0, 4, 7, 3, 8, 9]",1,0.333333,0.333333
4,apple,"[0, 4, 6, 8, 9]","[0, 1, 6, 7, 8, 10, 2, 3, 4, 5]",1,1.000000,0.666667
5,Blueberries Strawberries,"[5, 8, 11]","[2, 3, 4, 5, 6, 7, 0, 1, 8, 9]",1,0.000000,0.000000


In [71]:
def recall(relevant_doc_ids: list[list[int]], retrieved_doc_ids: list[list[int]], K):
    """ 
    no. of relevant docs in top K retrieved docs / no. of relevant docs
    """
    result = [0] * len(relevant_doc_ids)
    for i in range(len(relevant_doc_ids)):
        rel_docs_count = 0
        for doc_id in retrieved_doc_ids[i][:K]:
            if doc_id in relevant_doc_ids[i]:
                rel_docs_count += 1
        result[i] = rel_docs_count/len(relevant_doc_ids)

    return result, round(float(mean(result)), 3)


result, rec = recall(df['relevant_doc_ids'], df['retrieved_doc_ids'], K)
print(f"Result: {result}\nRecall: {rec}")

df[f"recall@{K}"] = result
df    

Result: [0.3333333333333333, 0.0, 0.3333333333333333, 0.16666666666666666, 0.3333333333333333, 0.0]
Recall: 0.194


,query,relevant_doc_ids,retrieved_doc_ids,success@3,mrr@3,precision@3,recall@3
0,apple banana,"[0, 4, 6, 9]","[0, 3, 6, 11, 9, 1, 2, 5, 7, 8]",1,1.000000,0.666667,0.333333
1,grapes,"[3, 7]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]",1,0.000000,0.000000,0.000000
2,banana mango,"[1, 4, 6, 10]","[2, 4, 6, 7, 9, 0, 1, 3, 5, 8]",1,0.000000,0.666667,0.333333
3,Cherry,"[2, 10, 11]","[5, 1, 2, 6, 0, 4, 7, 3, 8, 9]",1,0.333333,0.333333,0.166667
4,apple,"[0, 4, 6, 8, 9]","[0, 1, 6, 7, 8, 10, 2, 3, 4, 5]",1,1.000000,0.666667,0.333333
5,Blueberries Strawberries,"[5, 8, 11]","[2, 3, 4, 5, 6, 7, 0, 1, 8, 9]",1,0.000000,0.000000,0.000000


In [72]:
docs = []
for i, d in enumerate(dataset):
    docs.append({"relevant":d["relevant_doc_ids"], "retrieved": retrieved_docs[i]})

docs

[{'relevant': [0, 4, 6, 9], 'retrieved': [0, 3, 6, 11, 9, 1, 2, 5, 7, 8]},
 {'relevant': [3, 7], 'retrieved': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]},
 {'relevant': [1, 4, 6, 10], 'retrieved': [2, 4, 6, 7, 9, 0, 1, 3, 5, 8]},
 {'relevant': [2, 10, 11], 'retrieved': [5, 1, 2, 6, 0, 4, 7, 3, 8, 9]},
 {'relevant': [0, 4, 6, 8, 9], 'retrieved': [0, 1, 6, 7, 8, 10, 2, 3, 4, 5]},
 {'relevant': [5, 8, 11], 'retrieved': [2, 3, 4, 5, 6, 7, 0, 1, 8, 9]}]